# 0-Tokenizer

Tokenizer（分词器）在 NLP 领域扮演着基础且关键的作用，它将文本分割成单词或子词并转化为数组编号，为模型提供可处理的输入，在文本预处理、语义理解及适配不同语言和任务等方面奠定基础，是连接自然语言文本与计算机可处理数据的重要桥梁.

## 子词分词算法

常见的子词分词算法有三种：

1. 字节对编码（Byte Pair Encoding，BPE）
2. WordPiece
3. Unigram

### BPE

BPE 是一种简单的数据压缩技术，它会迭代地替换序列中最频繁出现的字节对。BPE 依赖一个预分词器，该预分词器会将训练数据分割成单词（在本项目中，我们使用按空格分词的方法作为预分词方法）.

在预分词之后，会创建一组唯一的单词，并确定它们在数据中的出现频率。接下来，BPE 会创建一个基础词表，该词表包含预分词器最初生成的数据中所有唯一单词的符号。然后，会将这对符号从词表中移除，新形成的符号将加入词表。在迭代过程中，BPE 算法会合并频繁出现的符号对.

给定词表的大小，BPE（字节对编码）算法最终会合并出现频率最高的符号对，直到收敛到该大小.

---

以语料`{"low":5, "lower":2, "newest":6, "widest":3}`为例：
1. 初始拆分：`l o w</w>:5`、`l o w e r</w>:2`、`n e w e s t</w>:6`、`w i d e s t</w>:3`
2. 第一轮：`e s`出现9次（最高），合并为`es`
3. 第二轮：`es t</w>`出现9次，合并为`est</w>`
4. 持续合并后，词表包含`l`、`o`、`w`、`low`、`est</w>`等单元
5. 编码生僻词`unwidest`：拆分为`u n + w + i + d + est</w>`，全部落入词表

---

### WordPiece

WordPiece 算法与 BPE 非常相似。WordPiece 首先将词表初始化为包含训练数据中出现的每个字符，然后逐步学习给定数量的合并规则. 与 BPE 不同的是，WordPiece 并不选择最频繁出现的符号对，而是选择那个加入词表后能使训练数据出现的可能性最大化的符号对.

---

1. **初始化**：和BPE一致，将所有单词拆分为最小单元（字符），添加词尾标记`##`（表示该子词出现在单词内部，如`play##ing`），统计每个单元的出现频次。
2. **迭代合并**（核心差异点）：
   - 对每一对相邻子词（a, b），计算「合并为新子词ab」后语料的对数似然增益：
     $$\text{score}(a,b) = \frac{\text{count}(ab)}{\text{count}(a) \times \text{count}(b)}$$
   - 选择得分最高的子词对合并为新子词，加入词表。
   - 重复上述步骤，直到词表达到预设大小（如30k）或得分低于阈值。
3. **输出**：最终词表 + 合并规则。

---

### Unigram

Unigram 算法将其基础词表初始化为大量的符号，然后逐步削减每个符号，以获得一个更小的词表。它会在训练数据上定义一个对数似然损失，以此来确定是否从词表中移除某个符号.

## 训练一个最简单的分词器

在本节中，我们将学习基于 transformers 库来训练你自己的分词器.

### 初始化

首先，我们应该初始化我们的分词器，并确定选择哪种方法。我们将使用字节对编码（BPE）算法.

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE()) # 创建一个基于BPE的分词器模型
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False) # 使用ByteLevel预分词器，不添加前缀空格

---

- `pre_tokenizer`: 分词器的“预分词”环节，作用是在核心BPE分词前，先对文本做初步拆分/处理。
- `pre_tokenizers.ByteLevel()`: 使用**字节级预分词器**（ByteLevel），这是RoBERTa/GPT系列的标配：
  - 核心特点：将文本编码为UTF-8字节（而非字符），能处理所有语言（包括中文、emoji等），无需担心OOV（未登录词）。
  - `add_prefix_space=False`: 关闭“给非首词添加前缀空格”的规则。默认`True`时，会给句子中除第一个词外的所有词加前缀空格（比如`"hello world"`会变成`"hello Ġworld"`），设置`False`则不添加。

---

### 定义特殊标记

数据集中存在一些我们不希望被分词的特殊标记，我们会将这些标记定义为特殊标记，并将它们传递给分词器训练器，以防止出现错误的分词情况.

In [2]:
special_tokens = ["<unk>", "<s>", "</s>"] # 定义特殊标记

trainer = trainers.BpeTrainer(
        vocab_size=256, # 词汇表大小
        special_tokens=special_tokens, # 添加特殊标记
        show_progress=True, # 显示训练进度
        initial_alphabet=pre_tokenizers.ByteLevel.alphabet() # 使用ByteLevel的初始字母表
)

### 从文件中读取数据

在本次实验中，我们使用 JSON Lines（jsonl）格式来存储 Tokenizer 训练数据，分词器内置的训练函数要求训练数据以迭代器的形式传入，因此，我们首先获取一个数据读取的生成器.

In [3]:
import json

def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

data_path = './toydata/tokenizer_data.jsonl'

In [4]:
data_iter = read_texts_from_jsonl(data_path)
print(f'Row 1: {next(data_iter)}')

Row 1: <s>近年来，人工智能技术迅速发展，深刻改变了各行各业的面貌。机器学习、自然语言处理、计算机视觉等领域的突破性进展，使得智能产品和服务越来越普及。从智能家居到自动驾驶，再到智能医疗，AI的应用场景正在快速拓展。随着技术的不断进步，未来的人工智能将更加智能、更加贴近人类生活。</s>


### 开始训练!

我们使用分词器的内置函数 `tokenizer.train_from_iterator` 来训练分词器.

---

`train_from_iterator`的核心特点：
- **输入**：接收任意可迭代对象（生成器、列表、迭代器等），逐批次读取文本，内存占用极低。
- **适用场景**：
  - 处理大文件（GB级JSONL/文本文件），无法一次性加载到内存；
  - 语料来自数据库/网络流等动态生成的文本；
  - 绝大多数工业级/实战场景。
- **优势**：内存友好、灵活、适配大规模数据，是官方文档优先推荐的方式。

---

In [5]:
tokenizer.train_from_iterator(data_iter, trainer=trainer) # 从数据迭代器中训练分词器

### 设置解码器

In [6]:
tokenizer.decoder = decoders.ByteLevel() # 使用ByteLevel解码器，以确保正确处理字节级别的编码和解码

接下来，检查一下特殊标记是否得到了妥善处理。

In [7]:
# 测试分词器，确保特殊标记已正确添加，条件为True时通过断言检查，否则抛出异常
assert tokenizer.token_to_id('<unk>') == 0
assert tokenizer.token_to_id('<s>') == 1
assert tokenizer.token_to_id('</s>') == 2

### 将训练好的分词器保存到磁盘

In [8]:
import os

tokenizer_dir = "./model/toy_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True) # 创建目录以保存分词器模型
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json")) # 保存分词器的JSON配置文件
tokenizer.model.save(tokenizer_dir) # 生成vocab.json和merges.txt文件

['./model/toy_tokenizer\\vocab.json', './model/toy_tokenizer\\merges.txt']

### 手动创建一份配置文件

In [9]:
config = {
    "add_bos_token": False, # 是否添加句子开始标记
    "add_eos_token": False, # 是否添加句子结束标记
    "add_prefix_space": False, # 是否在文本前添加空格
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>", # 未知标记
            "lstrip": False, # 是否去除左侧空格
            "normalized": False, # 是否归一化
            "rstrip": False, # 是否去除右侧空格
            "single_word": False, # 是否为单词标记
            "special": True # 是否为特殊标记
        },
        "1": {
            "content": "<s>", # 句子开始标记
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "</s>", # 句子结束标记
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [], # 额外的特殊标记
    "bos_token": "<s>", # 句子开始标记
    "clean_up_tokenization_spaces": False, # 是否清理分词空格
    "eos_token": "</s>", # 句子结束标记
    "legacy": True, # 兼容旧版本
    "model_max_length": 32768, # 模型最大长度
    "pad_token": "<unk>", # 填充标记
    "sp_model_kwargs": {}, # SentencePiece模型参数
    "spaces_between_special_tokens": False, # 特殊标记之间是否添加空格
    "tokenizer_class": "PreTrainedTokenizerFast", # 分词器类型为预训练快速分词器
    "unk_token": "<unk>", # 未知标记
    "chat_template": "{{ '<s>' + messages[0]['text'] + '</s>' }}" # 聊天模板为何种格式
}

with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4) # 保存分词器配置文件

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


现在我们已经训练了一个简单的分词器，并将其进行保存，接下来，我们试着加载它，并使用其帮助我们对文本进行编解码.

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./model/toy_tokenizer") # 加载保存的分词器模型
msg = [{"text": "失去的东西就要学着去接受，学着放下。"}]
new_msg = tokenizer.apply_chat_template( # 应用聊天模板
    msg,
    tokenize=False # 不进行分词
)
print(f'原始文本：{msg}')
print(f'修改文本：{new_msg} (添加自定义聊天模板)')

原始文本：[{'text': '失去的东西就要学着去接受，学着放下。'}]
修改文本：<s>失去的东西就要学着去接受，学着放下。</s> (添加自定义聊天模板)


In [11]:
print(f'分词器词表大小：{tokenizer.vocab_size}')

分词器词表大小：259


In [12]:
model_inputs = tokenizer(new_msg)
print(f'查看分词结果：\n{model_inputs}')

查看分词结果：
{'input_ids': [1, 164, 100, 112, 164, 239, 122, 166, 251, 229, 163, 119, 253, 167, 101, 126, 164, 111, 112, 167, 102, 226, 164, 258, 102, 166, 254, 225, 164, 239, 122, 165, 239, 101, 164, 240, 248, 174, 123, 237, 164, 258, 102, 166, 254, 225, 165, 245, 125, 163, 119, 236, 162, 225, 227, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


---

ByteLevel BPE（字节级字节对编码，简称BBPE）是**传统BPE的改进版**，是当前大语言模型（如GPT系列、LLaMA、Claude）的主流分词算法，核心是把文本转为UTF-8字节序列，在字节粒度上迭代合并高频相邻字节对，生成子词单元，彻底解决OOV（未登录词）问题，同时兼顾多语言、特殊符号与词表可控性。

### 一、核心定位与由来
- 起源：由OpenAI在GPT-2中首次提出并工程化落地，解决传统字符级BPE的多语言、OOV、特殊符号覆盖不足问题。
- 本质：把文本先转为**UTF-8字节流**（每个字符1–4字节），以 **字节（0x00–0xFF，共256种）** 为最小单元，再执行BPE的迭代合并逻辑，生成子词词表与合并规则。
- 优势：
  1.  零OOV：任何文本（中文、Emoji、数学符号、乱码）都能被UTF-8编码为字节，无未知符号。
  2.  多语言统一：不依赖特定语言字符集，天然支持中英混合、多语种混排。
  3.  词表可控：初始词表仅256个字节，合并后词表大小可按需设定（常见32K–128K），平衡语义粒度与计算效率。


### 二、核心流程（训练+推理）
#### 1. 训练阶段：学习合并规则与构建词表
1.  文本转字节：输入语料全部转为UTF-8字节序列，例如`"Hello 中😊"` → `[48, 65, 6C, 6C, 6F, 20, E4, B8, AD, F0, 9F, 98, 8A]`。
2.  初始化词表：包含全部256个字节值（0x00–0xFF），无额外预设字符。
3.  迭代合并（核心）：
    - 统计所有**相邻字节对**的出现频次；
    - 合并**频次最高**的字节对为一个新子词token；
    - 用新token替换语料中所有该字节对，更新序列；
    - 重复统计→合并→替换，直到达到预设词表大小或合并次数上限。
4.  输出：最终词表（字节+合并生成的子词）+ 合并规则列表（按合并优先级排序）。

#### 2. 推理阶段：用训练好的规则分词
1.  新文本先转为UTF-8字节序列；
2.  按**合并规则优先级**，从高到低扫描并合并可匹配的字节对，直到无法合并；
3.  输出最终token序列，完成分词。


### 三、与传统字符级BPE的关键区别
| 维度 | 传统字符级BPE | ByteLevel BPE |
|------|---------------|---------------|
| 最小单元 | 字符（依赖语言字符集） | 字节（0x00–0xFF，共256种） |
| OOV问题 | 易出现（生僻字、Emoji、特殊符号易成UNK） | 零OOV（所有文本均可编码为字节） |
| 多语言支持 | 差，需适配不同语言字符集 | 优，天然统一处理所有语言 |
| 初始词表 | 随语言字符集变大（如中文数千） | 固定256，极小且通用 |
| 典型应用 | 早期NLP模型、单语言任务 | GPT系列、LLaMA、Claude等现代大模型 |

### 四、最直观的实际例子（中文+英文+符号）
直接看**单字/字符→UTF-8字节→十六进制/十进制**的对应关系，这是BBPE处理文本的第一步，也是最关键的一步：
| 原始字符 | UTF-8字节数 | 十六进制字节序列 | 十进制字节序列（BBPE最小单元） |
|----------|-------------|------------------|--------------------------------|
| 我       | 3           | 0xE6 0x88 0x91   | [230, 136, 145]                |
| 爱       | 3           | 0xE7 0x88 0xB1   | [231, 136, 177]                |
| 你       | 3           | 0xE4 0xBD 0xA0   | [228, 189, 160]                |
| a        | 1           | 0x61             | [97]                           |
| ！       | 3           | 0xEF 0xBC 0x81   | [239, 188, 129]                |
| 😊       | 4           | 0xF0 0x9F 0x98 0x8A | [240, 159, 152, 138]        |

---

---

### token_type_ids：文本段的“分类标签”
#### 1. 核心作用
`token_type_ids`的官方名称是“段落ID/类型ID”，核心目的是**让模型区分输入中的不同文本片段**，仅用于**多文本任务**（比如：
- 问答任务：区分“问题段”和“答案段”；
- 文本匹配：区分“句子A”和“句子B”；
- 摘要任务：区分“原文”和“摘要”）。

#### 2. 结果解读
此处的`token_type_ids`全是`0`，原因是：
```
[0, 0, 0, ..., 0]  # 所有位置都是0
```
- 输入是**单文本**（只有`<s>失去的东西就要学着去接受，学着放下。</s>`这一段），没有多个文本段需要区分，因此分词器默认给所有Token分配`0`；
- 如果是多文本输入（比如`["你好", "我很好"]`），分词器会给第一段标`0`、第二段标`1`，示例：
  ```
  input_ids:      [1, xx, xx, 2, 1, yy, yy, 2]  # 两段文本
  token_type_ids: [0, 0, 0, 0, 1, 1, 1, 1]      # 第一段0，第二段1
  ```

#### 3. 对LLM的意义
场景是单文本生成（比如聊天模板处理后的单句），`token_type_ids`**完全没用**——模型训练/推理时会忽略这个字段，但分词器依然会生成它，是为了兼容Hugging Face的统一输入格式（避免因缺少字段报错）。


### attention_mask：模型的“注意力过滤器”
#### 1. 核心作用
Transformer模型的核心是“自注意力机制”，会对每个Token计算注意力权重。`attention_mask`的作用是：
- `1`：标记“这是有效Token，模型需要关注、计算注意力”；
- `0`：标记“这是填充（pad）Token，模型要忽略、不计算注意力”。

#### 2. 结果解读
此处的`attention_mask`全是`1`，原因是：
```
[1, 1, 1, ..., 1]  # 所有位置都是1
```
- 输入文本没有被“填充”（长度刚好等于Token数），所有Token都是有效文本，因此全标`1`；
- 只有**批量处理文本**时，这个字段才会体现价值：比如一批输入有3句话，长度分别是10、15、20，需要把所有句子填充到20个Token（最短的句子补9个`<pad>`），此时：
  ```
  # 长度10的句子填充后
  input_ids:      [1, xx, ..., 2, 0, 0, ..., 0]  # 后10个是<pad>的ID（0）
  attention_mask: [1, 1, ..., 1, 0, 0, ..., 0]  # 后10个标0，模型忽略
  ```

#### 3. 为什么必须存在？
如果没有`attention_mask`，模型会把填充的`<pad>`Token当成有效文本计算注意力，导致：
- 计算结果错误（填充字符无语义，却参与了注意力计算）；
- 模型学到无意义的规律（比如“填充字符和文本有关联”）；
- 甚至引发维度错误（批量输入要求长度一致，填充是必须的）。

---

In [13]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=False) # 对分词结果进行解码，保留特殊字符
print(f'对分词结果进行解码：{response} (保留特殊字符)' )

对分词结果进行解码：<s>失去的东西就要学着去接受，学着放下。</s> (保留特殊字符)


In [14]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=True) # 对分词结果进行解码，跳过特殊字符
print(f'对分词结果进行解码：{response} (移除特殊字符)' )

对分词结果进行解码：失去的东西就要学着去接受，学着放下。 (移除特殊字符)


## 参考资料

- [Hugging Face NLP Course](https://huggingface.co/learn/nlp-course/zh-CN/chapter2/4)